**Low Risk Investments by Sector**

In [1]:
# Provides ways to work with large multidimensional arrays
import numpy as np 
# Allows for further data manipulation and analysis
import pandas as pd
from pandas_datareader import data as web # Reads stock data 
import matplotlib.pyplot as plt # Plotting
import matplotlib.dates as mdates # Styling dates
%matplotlib inline

import datetime as dt # For defining dates
import mplfinance as mpf # Matplotlib finance

import time

# Used to get data from a directory
import os
from os import listdir
from os.path import isfile, join

**Dates & Other Constants**

In [2]:
# Define path to files

PATH = "../raw_data/data_stocks/"

# Start date defaults
S_YEAR = 2019
S_MONTH = 1
S_DAY = 3
S_DATE_STR = "2019-01-03"
S_DATE_DATETIME = dt.datetime(S_YEAR, S_MONTH, S_DAY)

# End date defaults
E_YEAR = 2022
E_MONTH = 4
E_DAY = 5
E_DATE_STR = "2022-04-05"
E_DATE_DATETIME = dt.datetime(E_YEAR, E_MONTH, E_DAY)


**Get Stock File Names in a List**

In [3]:
files = [x for x in listdir(PATH) if isfile(join(PATH, x))]
tickers = [os.path.splitext(x)[0] for x in files]
tickers

# On MacOS Only
#tickers.remove('.DS_Store')
tickers.sort()

In [4]:
len(tickers)

2896

Function that Returns a Dataframe from a CSV

In [5]:
def get_df_from_csv(ticker):
    try:
        df = pd.read_csv(PATH + ticker + '.csv')
    except FileNotFoundError:
        pass
        # print("File Doesn't Exist")
    else:
        return df

Function that Saves Dataframe to CSV

In [6]:
def save_dataframe_to_csv(df, ticker):
    df.to_csv(PATH + ticker + '.csv')

Return 1st Valid Date in Dataframe

In [7]:
def get_valid_dates(df, sdate, edate):  
    try:
        mask = (df['Date'] > sdate) & (df['Date'] <= edate) 
        sm_df = df.loc[mask]
        sm_df = sm_df.set_index(['Date'])
        sm_date = sm_df.index.min()
        last_date = sm_df.index.max()
        date_leading = '-'.join(('0' if len(x)<2 else '')+x for x in sm_date.split('-'))
        date_ending = '-'.join(('0' if len(x)<2 else '')+x for x in last_date.split('-'))
    except Exception:
        print("Date Corrupted")
    else:
        return date_leading, date_ending

Returns Return on Investment over Time

In [8]:
def roi_between_dates(df, sdate, edate):
    try: 
        start_val = df.loc[sdate,'Adj Close'] 
        end_val = df.loc[edate,'Adj Close']
        roi = ((end_val - start_val) / start_val)
    except Exception:
        print("Data Corrupted")
    else:
        return roi

Get Mean Between Dates

In [9]:
def get_mean_between_dates(df, sdate, edate):
    mask = (df['Date'] > sdate) & (df['Date'] <= edate)
    return df.loc[mask]["Adj Close"].mean()

Get Standard Deviation Between Dates

In [10]:
def get_sd_between_dates(df, sdate, edate):
    mask = (df['Date'] > sdate) & (df['Date'] <= edate)
    return df.loc[mask]["Adj Close"].std()

Get Coefficient of Variation Between Dates

In [11]:
def get_cov_between_dates(df, sdate, edate):
    mean = get_mean_between_dates(df, sdate, edate)
    sd = get_sd_between_dates(df, sdate, edate)
    return sd / mean

Merge Multiple Stocks in One Dataframe by Column Name

In [12]:
def merge_df_by_column_name(col_name, sdate, edate, *tickers):
    # Will hold data for all dataframes with the same column name
    mult_df = pd.DataFrame()
    
    for x in tickers:
        df = get_df_from_csv(x)
        df['Date'] = pd.to_datetime(df['Date'])
        # Use a mask to grab data between defined dates
        mask = (df['Date'] >= sdate) & (df['Date'] <= edate)
        mult_df[x] = df.loc[mask][col_name]
        
    return mult_df

11 Stock Market Sectors

**Technology :** Manufacturing of electronics, software, or related to information technology

**Health Care :** Biotech, hospitals, medical devices, drugs

**Financials :** Banks, investment funds, and insurance

**Real Estate :** Residential, industrial, and retail real estate

**Energy :** Production and supply of energy

**Materials :** Mining, refining, chemical, and forestry

**Consumer Discretionary :** Retailers, apparel, media, durables and services

**Industrials :** Construction, machinery, fabrication, manufacturing, defense, and aerospace

**Utilities :**  Direct providers of electric, gas, and water

**Consumer Staples :** Food, beverage as well as products consumers deem essential

**Telecommunication :** Cable, internet providers, wireless, and satellite

Get Sector Stocks

In [13]:
# For Mac
sec_df = pd.read_csv('../raw_data/stocks_ticker/stock_sectors.csv')
sec_df

# Get Industrials DF
indus_df = sec_df.loc[sec_df['Sector'] == "Industrials"]
health_df = sec_df.loc[sec_df['Sector'] == "Health Care"]
it_df = sec_df.loc[sec_df['Sector'] == "Information Technology"]
comm_df = sec_df.loc[sec_df['Sector'] == "Communication Services"]
staple_df = sec_df.loc[sec_df['Sector'] == "Consumer Staples"]
discretion_df = sec_df.loc[sec_df['Sector'] == "Consumer Discretionary"]
utility_df = sec_df.loc[sec_df['Sector'] == "Utilities"]
financial_df = sec_df.loc[sec_df['Sector'] == "Financials"]
material_df = sec_df.loc[sec_df['Sector'] == "Materials"]
restate_df = sec_df.loc[sec_df['Sector'] == "Real Estate"]
energy_df = sec_df.loc[sec_df['Sector'] == "Energy"]

Returns a DF with ROIs for all Stocks

In [14]:
def get_rois_for_stocks(stock_df):
    # Will hold all tickers & stock rois
    tickers = []
    rois = []

    # iterrows provides the index and column for each row in the DF
    for index, row in stock_df.iterrows():
        df = get_df_from_csv(row['Symbol'])
    
        # If we can't find the ticker delete it from the dataframe
        if df is None:
            pass
            # print(row['Symbol'], " is not available")
        else:
            tickers.append(row['Symbol'])
            sdate, edate = get_valid_dates(df, '2018-01-01', '2020-12-31')
            df = df.set_index(['Date'])
            roi = roi_between_dates(df, sdate, edate)
            rois.append(roi)
    return pd.DataFrame({'Ticker':tickers, 'ROI':rois})

In [15]:
industrial = get_rois_for_stocks(indus_df)
health_care = get_rois_for_stocks(health_df)
it = get_rois_for_stocks(it_df)
commun = get_rois_for_stocks(comm_df)
staple = get_rois_for_stocks(staple_df)
discretion = get_rois_for_stocks(discretion_df)
utility = get_rois_for_stocks(utility_df)
finance = get_rois_for_stocks(financial_df)
material = get_rois_for_stocks(material_df)
restate = get_rois_for_stocks(restate_df)
energy = get_rois_for_stocks(energy_df)

Top Industrials

In [16]:
industrial.sort_values(by=['ROI'], ascending=False).head(5)

,Ticker,ROI
24,GNRC,3.459894
16,ETN,2.113822
10,CPRT,1.668834
44,PWR,1.429004
40,ODFL,1.390986


Top Healthcare

In [17]:
health_care.sort_values(by=['ROI'], ascending=False).head(5)

,Ticker,ROI
16,CTLT,2.366872
25,DXCM,2.209932
59,WST,1.962799
33,IDXX,1.739614
4,ALGN,1.643875


Top Communications

In [18]:
commun.sort_values(by=['ROI'], ascending=False).head(10)

,Ticker,ROI
4,CHTR,1.297128
19,TMUS,1.066350
15,NFLX,1.020212
10,FB,1.013267
0,ATVI,1.003220
20,TTWO,0.997789
22,TWTR,0.879556
9,EA,0.789117
21,DIS,0.682979
2,GOOG,0.675078


Top Staples

In [19]:
staple.sort_values(by=['ROI'], ascending=False).head(5)

,Ticker,ROI
10,EL,1.051833
9,COST,0.923046
22,MNST,0.896637
7,CAG,0.801374
11,GIS,0.647461


Top Discretionary

In [20]:
discretion.sort_values(by=['ROI'], ascending=False).head(5)

,Ticker,ROI
51,TSLA,10.377370
41,PENN,3.440617
17,ETSY,2.785319
10,CMG,2.127729
50,TGT,1.797054


Top Utilities

In [21]:
utility.sort_values(by=['ROI'], ascending=False).head(5)

,Ticker,ROI
18,NEE,0.899435
4,AWK,0.787637
0,AES,0.770642
25,SO,0.533028
27,XEL,0.457615


Top Finance

In [22]:
finance.sort_values(by=['ROI'], ascending=False).head(5)

,Ticker,ROI
39,MSCI,2.100301
34,MKTX,1.760462
37,MCO,1.098857
51,SIVB,0.980341
49,SPGI,0.970903


Top Materials

In [23]:
material.sort_values(by=['ROI'], ascending=False).head(5)

,Ticker,ROI
11,FCX,1.567112
3,BLL,1.119669
1,ALB,0.959942
22,SHW,0.907899
17,NEM,0.839972


Top Real Estate

In [24]:
restate.sort_values(by=['ROI'], ascending=False).head(5)

,Ticker,ROI
8,EQIX,1.112940
17,PLD,0.841405
21,SBAC,0.771372
0,ARE,0.688283
7,DRE,0.684079


Top Energy

In [25]:
energy.sort_values(by=['ROI'], ascending=False).head(5)

,Ticker,ROI
8,HES,0.300170
19,WMB,0.030915
9,KMI,-0.026720
16,PXD,-0.122947
15,PSX,-0.133590


Portfolio List

In [44]:
port_list = ["GNRC", "DXCM", "AMD", "NFLX", "COST", "TGT", "AES", "MSCI", 
             "NEM", "SBAC", "HES"]

Check for Correlation

In [45]:
mult_df = merge_df_by_column_name("daily_return",  "2019-01-03", "2022-04-05", *port_list)
mult_df

# Generate a Correlation Matrix
mult_df.corr()

,GNRC,DXCM,AMD,NFLX,COST,TGT,AES,MSCI,NEM,SBAC,HES
GNRC,1.000000,0.362135,0.410602,0.314682,0.390427,0.348294,0.432668,0.488260,0.161205,0.382980,0.294268
DXCM,0.362135,1.000000,0.380586,0.335258,0.287725,0.219398,0.284324,0.520413,0.100859,0.342548,0.209680
AMD,0.410602,0.380586,1.000000,0.430986,0.434194,0.337098,0.287795,0.526674,0.128252,0.336368,0.253388
NFLX,0.314682,0.335258,0.430986,1.000000,0.382680,0.259088,0.225051,0.434027,0.061184,0.234332,0.105305
COST,0.390427,0.287725,0.434194,0.382680,1.000000,0.477781,0.328815,0.491728,0.163665,0.459252,0.127885
TGT,0.348294,0.219398,0.337098,0.259088,0.477781,1.000000,0.313069,0.339650,0.075922,0.305729,0.213917
AES,0.432668,0.284324,0.287795,0.225051,0.328815,0.313069,1.000000,0.476271,0.179486,0.314490,0.447509
MSCI,0.488260,0.520413,0.526674,0.434027,0.491728,0.339650,0.476271,1.000000,0.197723,0.510727,0.318149
NEM,0.161205,0.100859,0.128252,0.061184,0.163665,0.075922,0.179486,0.197723,1.000000,0.276635,0.145001
SBAC,0.382980,0.342548,0.336368,0.234332,0.459252,0.305729,0.314490,0.510727,0.276635,1.000000,0.200501


In [46]:
# Get the number of samples
days = len(mult_df.index)
days

819

In [47]:
def get_stock_price_on_date(ticker, date):
    df = get_df_from_csv(ticker)
    df = df.set_index(['Date'])
    return df.loc[date,'Adj Close']

In [48]:
# 01-04-2021 is the 1st date in 2021
investment_date = '2021-01-04'
print("GNRC $", get_stock_price_on_date('GNRC', investment_date)) # 2 449.12
print("DXCM $", get_stock_price_on_date('DXCM', investment_date)) # 1 357.74
print("AMD $", get_stock_price_on_date('AMD', investment_date)) # 4 369.2
print("NFLX $", get_stock_price_on_date('NFLX', investment_date)) # 1 522.86
print("COST $", get_stock_price_on_date('COST', investment_date)) # 1 377.88
print("TGT $", get_stock_price_on_date('TGT', investment_date)) # 3 527.46
print("AES $", get_stock_price_on_date('AES', investment_date)) # 18 410.76
print("MSCI $", get_stock_price_on_date('MSCI', investment_date)) # 1 435.40
print("NEM $", get_stock_price_on_date('NEM', investment_date)) # 7 434.49
print("SBAC $", get_stock_price_on_date('SBAC', investment_date)) # 2 548.84
print("HES $", get_stock_price_on_date('HES', investment_date)) # 9 472.23

GNRC $ 224.5599975585937
DXCM $ 357.739990234375
AMD $ 92.3000030517578
NFLX $ 522.8599853515625
COST $ 376.7002258300781
TGT $ 174.46778869628906
AES $ 22.52255249023437
MSCI $ 433.8714294433594
NEM $ 60.45637893676758
SBAC $ 272.917724609375
HES $ 51.91611862182617


In [49]:
# Total Investment
tot_inv = 449.12 + 357.74 + 369.2 + 522.86 + 377.88 + 527.46 + 410.76 + \
435.40 + 434.49 + 548.84 + 472.23
tot_inv

4905.980000000001

Returns Portfolio Weight

In [50]:
def get_port_weight(price, total):
    return price / total

In [51]:
gnrc_w = get_port_weight(449.12, 4905.98)
dxcm_w = get_port_weight(357.74, 4905.98)
amd_w = get_port_weight(369.2, 4905.98)
nflx_w = get_port_weight(522.86, 4905.98)
cost_w = get_port_weight(377.88, 4905.98)
tgt_w = get_port_weight(527.46, 4905.98)
aes_w = get_port_weight(410.76, 4905.98)
msci_w = get_port_weight(435.40, 4905.98)
nem_w = get_port_weight(434.49, 4905.98)
sbac_w = get_port_weight(548.84, 4905.98)
hes_w = get_port_weight(472.23, 4905.98)

In [52]:
wts = np.array([gnrc_w, dxcm_w, amd_w, nflx_w, cost_w, tgt_w, aes_w, msci_w, \
                nem_w, sbac_w, hes_w])
wts

array([0.09154542, 0.07291917, 0.0752551 , 0.10657606, 0.07702437,
       0.10751369, 0.08372639, 0.08874883, 0.08856335, 0.11187163,
       0.096256  ])

In [53]:
port_var = np.dot(wts.T, np.dot(mult_df.cov() * 756, wts))
print("Portfolio Covariance :", port_var)
print("GNRC Var :", mult_df["GNRC"].var() * 756)
print("NFLX Var :", mult_df["NFLX"].var() * 756)

Portfolio Covariance : 0.17252017872963393
GNRC Var : 0.5895197678005564
NFLX Var : 0.5099425113690421


Diversifiable Risk

In [54]:
# Diversifiable Risk = Portfolio Variance - All Weighted Variances
def calc_diversifiable_risk(df, tickers, weights):
    # Gets number of days
    days = len(mult_df.index)
    # Calculate covariance of portfolio
    port_covar = np.dot(weights.T, np.dot(df.cov() * days, weights)) 
    
    i = 0
    while i < len(tickers):
        wt_sq = weights[i] ** 2
        stk_var = mult_df[tickers[i]].var() * days
        wt_var = wt_sq * stk_var
        port_covar = port_covar - wt_var
        i += 1
    return port_covar

In [55]:
div_risk = calc_diversifiable_risk(mult_df, port_list, wts)
div_risk

0.13832350542611876

Find Risk that Can't be Diversified

In [56]:
print("Systematic Risk :", (port_var - div_risk))

Systematic Risk : 0.03419667330351517
